In [ ]:
### Preparation of packages

!pip install selfcheckgpt
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_lg-0.5.3.tar.gz


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
  Created wheel for selfcheckgpt: filename=selfcheckgpt-0.1.4-py3-none-any.whl size=12240 sha256=e0b7de12403779ae12e933daef46904775918c41943a81250d709566673ab5c9
  Stored in directory: /root/.cache/pip/wheels/dc/a6/76/107247d4924cb23f6e1b1987413b2a191979767e1e2c1d5751
Successfully built selfcheckgpt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3

In [ ]:
import re
import os
import csv
import json
from typing import List
import yaml

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu

import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
### import python packages

import nltk
import spacy
import scispacy
import numpy as np
from selfcheckgpt.modeling_selfcheck import SelfCheckBERTScore
from nltk.translate.meteor_score import single_meteor_score

# download wordnet
nltk.download('wordnet')

# Use SpaCy to extract clinical related entities
nlp = spacy.load("en_core_sci_lg")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/drive/MyDrive/Capstone project/Ethics/combined_file.xlsx'
xls = pd.ExcelFile(file_path)

# Initialize an empty DataFrame for hallucination scores
hallucination_scores_df = pd.DataFrame()

In [ ]:
### Hallucination Score

# 1 - precision
# 1 - TP / (TP+FP
# TP: common entities between generated text and label text
# use this to estimate hallucination

def Hallucination_Score(pred,label):
    doc1, doc2 = nlp(pred), nlp(label)
    x, y = [str(d) for d in doc1.ents], [str(d) for d in doc2.ents]
    if (len(x) == 0):
        return 1
    else:
        return 1 - len(list(set(list(x))&set(list(y))))/len(set(list(x)))

In [ ]:
# Iterate over each sheet in the Excel file
for sheet in xls.sheet_names:
    data = pd.read_excel(file_path, sheet_name=sheet)
    scores = data.apply(lambda row: Hallucination_Score(row['model_generated'], row['correct_summary']), axis=1)
    hallucination_scores_df[sheet] = scores

In [ ]:
hallucination_scores_df

,Falcon_output,GPT3,llama2_output,MedAlpaca_output,Mistral_output
0,1.000000,0.857143,0.946667,0.960000,0.884615
1,0.789474,0.700000,0.800000,1.000000,0.731707
2,1.000000,0.744186,0.879310,0.921053,0.846154
3,1.000000,0.789474,0.800000,1.000000,0.795455
4,1.000000,0.956522,0.916667,1.000000,0.878788
...,...,...,...,...,...
95,0.928571,0.742857,0.906250,0.962963,0.887324
96,1.000000,0.809524,0.916667,0.875000,0.652174
97,0.891304,0.846154,0.902439,0.902439,0.870968
98,1.000000,0.815789,0.894737,1.000000,0.906250


In [ ]:
# Define the path for the new Excel file
output_file_path = 'hallucination_scores.xlsx'

# Save the DataFrame to an Excel file
hallucination_scores_df.to_excel(output_file_path, index=False)

# The hallucination scores are now saved in 'hallucination_scores.xlsx'